In [2]:
import os
import re
import ast
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict, List, Dict, Literal, Any, Callable
from serpapi import SerpApiClient

load_dotenv()

class TextBlock(TypedDict):
    type: Literal["text"]
    text: str

class Message(TypedDict):
    role: Literal["system", "user", "assistant", "tool"]
    content: List[TextBlock]   

## AutoGen

业务目标：开发一个功能明确的web端应用：实时显示比特币的价格

智能体团队角色：
- ProductManager
- Engineer
- CodeReviewer
- UserProxy

1) 构建OpenAI模型客户端

In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai._model_info import ModelInfo, ModelFamily
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
GLM_4_1V_THINKING_FLASH_INFO = ModelInfo(
    family=ModelFamily.GPT_4,
    vision=True,
    function_calling=True,
    json_output=True,            # ✅ 必须字段（新版本）
    max_context_length=128000,
)
# 定义OpenAI的客户端
def create_openai_model_client():
    return OpenAIChatCompletionClient(
        model = os.getenv("LLM_MODEL_ID"),
        api_key = os.getenv("LLM_API_KEY"),
        base_url = os.getenv("LLM_BASE_URL"),
        model_info=GLM_4_1V_THINKING_FLASH_INFO
    )

# 定义智能体角色

#ProductManager产品经理
def create_product_manager(model_client):
    '''
    1. 负责启动整个流程
    2. 系统消息不仅定义了其职责，还规范了输出的结构，并包含了引导对话转向下一环节（工程师）的明确指令
    '''
    system_message = """
角色：你是一位经验丰富的产品经理，专门负责软件产品的需求分析和项目规划。
核心职责：
    1. **需求分析**：深入理解用户的需求，识别核心功能以及边界条件
    2. **技术规划**：基于需求制定清晰的技术实现路径
    3. **风险评估**：识别潜在的技术风险和用户体验问题
    4. **协调沟通**：与工程师以及其他团队成员继续有效沟通

当接到开发任务时，请按照以下的结构进行分析：
    1. 需求理解与分析
    2. 功能模块划分
    3. 技术选型建议
    4. 实现优先级排序
    5. 验收标准定义

请简洁明了地回应，并在分析完成后说“请工程师开始实现。”
    """
    return AssistantAgent(
        name="ProductManager",
        model_client=model_client,
        system_message=system_message
    )

# Engineer工程师

def create_engineer(model_client):
    '''
    1. 工程师系统消息聚焦于技术实现
    2. 列举工程师的技术专长
    3. 规定了其在接受到任务后的具体行动步骤
    4. 包含引导流程转向代码审查员的指令
    '''
    system_message = """
角色：你是一位资深的软件工程师。擅长python开发和Web应用构建。
你的技术专长包括：
    1. **Python 编辑**：熟练掌握 Python 的语法和最佳实践
    2. **Web 开发**：精通 Streamlit，Flask，Django 等架构
    3. **API 集成**：有丰富的第三方 API 集成经验
    4. **错误处理**：注重代码的健壮性和异常处理

当收到开发任务时，请：
    1. 仔细分析技术需求
    2. 选择合适的技术方案
    3. 编写完整的代码实现
    4. 添加必要的注释和说明
    5. 考虑边界情况和异常处理


    请提供完整的可运行代码，并在完成后说“请代码审查员检查”。

"""
    return AssistantAgent(
        model_client=model_client,
        name="Engineer",
        system_message=system_message
    )

# CodeReviewer 代码审查员
def create_code_reviewer(model_client):
    """创建代码审查员智能体
    1. 代码审查员侧重于代码的质量，安全性和规范性
    2. 系统消息详细列出审查的重点和流程，确保交付代码质量过关
    """
    system_prompt = """
角色：你是一个经验丰富的代码审查专家，专注于代码质量和最佳实践。
你的审查重点包括：
    1. **代码质量**：检查代码的可读性，可维护性和性能
    2. **安全性**：识别潜在的安全漏洞和风险点
    3. **最佳实践**：确保代码遵循行业标准和最佳实践
    4. **错误处理**：验证异常处理的完整性和合理性

审查流程：
    1. 仔细阅读和理解代码的逻辑
    2. 检查代码规范和最佳实践
    3. 识别潜在的问题和改进点
    4. 提供具体的修改建议
    5. 评估代码的整体质量

请提供具体的审查意见，完成后说“代码审查完成，请用户代理测试。”
"""
    return AssistantAgent(
        name="CodeReviewer",
        model_client=model_client,
        system_message=system_prompt
    )

# UserProxy 用户代理
def create_user_proxy():
    """创建用户代理智能体
    1. UserProxyAgent是一个特殊的智能体，它不依赖LLM进行回复，而是作为用户在系统中的代理。
    2. 他的 description 字段清晰地描述了其职责。
    3. 负责在人物最终完成后发出 TERMINATE 指令，以正常结束整个协作流程。
    """
    return UserProxyAgent(
        name="UserProxy",
        description="""
角色：用户代理
职责：
    1. 代理用户提出开发需求
    2. 执行最终的代码实现
    3.验证功能是否符合预期
    4. 提供用户反馈和建议

完成测试后请回复 TERMINATE。
"""
    )

软件开发的流程是相对固定的（需求->编码->审查->测试），因此 RoundRobinGroupChat (轮询群聊) 是理想的选择。我们按照业务逻辑顺序，将四个智能体加入到参与者列表中。

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
import asyncio

from autogen_agentchat.messages import TextMessage

def render_event(event):
    """
    将 AutoGen 事件对象渲染为人类可读的控制台输出
    """
    # 只关心文本消息（你现在的需求）
    if isinstance(event, TextMessage):
        role = event.source
        content = event.content.strip()

        print(f"---------- TextMessage ({role}) ----------")
        print(content)
        print()

    # 可选：过滤掉 ThoughtEvent（内部思考）
    else:
        # 如果你想看调试信息，可以打开
        # print(f"[{event.type}] from {event.source}")
        pass

async def run_software_development_team(task):
    print("🔧 正在初始化模型客户端...")
    llm_client = create_openai_model_client()

    print("👥 正在创建智能体团队...")
    product_manager = create_product_manager(llm_client)
    engineer = create_engineer(llm_client)
    code_reviewer = create_code_reviewer(llm_client)
    user_proxy = create_user_proxy()

    print("🚀 启动 AutoGen 软件开发团队协作...")
    print("=" * 60)
    kwargs = {
        "participants": [
            product_manager,
            engineer,
            code_reviewer,
            user_proxy
        ],
        "termination_condition": TextMentionTermination("TERMINATE"),
        "max_turns": 20
    }
    team_chat = RoundRobinGroupChat(**kwargs)
    
    final_event = None

    async for event in team_chat.run_stream(task=task):
        render_event(event)
        final_event = event
        
    print("=" * 60)
    print("✅ 团队协作完成！")
    return final_event

# 定义任务描述
task = """我们需要开发一个比特币价格显示应用，具体要求如下：
            核心功能：
            - 实时显示比特币当前价格（USD）
            - 显示24小时价格变化趋势（涨跌幅和涨跌额）
            - 提供价格刷新功能

            技术要求：
            - 使用 Streamlit 框架创建 Web 应用
            - 界面简洁美观，用户友好
            - 添加适当的错误处理和加载状态

            请团队协作完成这个任务，从需求分析到最终实现。"""

await run_software_development_team(task)

🔧 正在初始化模型客户端...
👥 正在创建智能体团队...
🚀 启动 AutoGen 软件开发团队协作...


/opt/homebrew/Caskroom/miniconda/base/envs/research-agent/lib/python3.11/site-packages/autogen_ext/models/openai/_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


---------- TextMessage (user) ----------
我们需要开发一个比特币价格显示应用，具体要求如下：
            核心功能：
            - 实时显示比特币当前价格（USD）
            - 显示24小时价格变化趋势（涨跌幅和涨跌额）
            - 提供价格刷新功能

            技术要求：
            - 使用 Streamlit 框架创建 Web 应用
            - 界面简洁美观，用户友好
            - 添加适当的错误处理和加载状态

            请团队协作完成这个任务，从需求分析到最终实现。

---------- TextMessage (ProductManager) ----------
以下是按照要求的分析过程：  


### 1. 需求理解与分析  
深入分析可知，该比特币价格显示应用的核心需求是**为用户提供比特币实时价格、24小时价格趋势及交互式刷新功能**；技术层面需满足用Streamlit构建Web应用、保证界面友好与美观、添加错误处理和加载状态的要求。需重点关注数据实时性、界面易用性及系统稳定性。  


### 2. 功能模块划分  
将应用拆分为四个核心模块，：  
- **数据获取模块**：通过API接口获取比特币实时价格、24小时历史价格数据；  
- **数据处理模块**：对原始数据进行清洗，计算“涨跌幅”“涨跌额”，并格式化数据用于展示；  
- **前端展示模块**：用Streamlit构建Web界面，展示实时价格、趋势图表等信息；  
- **交互控制模块**：实现“价格刷新”按钮功能，处理用户交互并触发数据更新流程。  

### 3. 技术选型建议  
- 框架：选用Streamlit，其支持Python代码直接渲染界面，能快速搭建Web应用，符合“简洁高效”的技术要求；  
- 数据源：选择CoinGecko/CoinMarketCap等权威API，保障数据实时性与可靠性；  
- 可视化：结合Pandas处理数据、Plotly/Matplotlib生成24小时趋势图表（Streamlit可无缝集成这些库）；  
- 异常处理：在各数据请求、解析环节添加try - exce